In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `C:\projects\julia-intro2\Project.toml`


# Intro to Julia

## Krisztián Schäffer

This is a fork of the notebook created by Robin Deits (Detroit Tech Watch May 26, 2020) https://tinyurl.com/tech-watch-julia [1]

[1] https://github.com/rdeits/DetroitTechWatch2020.jl/blob/master/Intro%20to%20Julia.ipynb

# Overview

* What is Julia?
* Tour of Julia
  * Functions, types, multiple dispatch
* Julia is fast! 
  * Benchmarks
* Bonus Features of Julia
  * Async tasks, multiprocessing, and metaprogramming
* What's hard to do in Julia?
* Essential Julia packages and tools

# What is Julia

<https://docs.julialang.org/en/v1/>

* Julia is a dynamic language like Python with the performance of a fast language like C
* Julia is a great choice for scientific computing, with:
  * Excellent performance
  * N-dimensional arrays
  * Parallel and distributed computing
* And it's also a nice environment for general purpose programming, with:
  * An active ecosystem of packages and good tools for managing them
  * A rich type system
  * Iterators, asynchronous tasks, metaprogramming, etc.


# Julia at a Glance

* First public release in 2012, version 1.0 released in 2018
* Free
    * Julia itself is BSD licensed
    * It bundles some GPLed libraries (which can be disabled if desired)
* Built-in ~~JIT~~ JAOT compiler transforms Julia code to native assembly at run time
  * Uses LLVM under the hood
* Garbage collected
* Dynamically typed
* Organized via multiple dispatch

# A Tour of Julia

##  The Basics

Arithmetic:

In [2]:
2 + 2

4

Strings:

In [3]:
# Strings
println("hello world")

hello world


Arrays:

In [4]:
x = [1, 2, 3, 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

## Functions

In [5]:
function say_hello(name)
    println("hello ", name)
end

say_hello (generic function with 1 method)

In [6]:
say_hello("world")

hello world


By default, a function is generic, so you can pass in any type you want:

In [7]:
say_hello([1, 2, 3])

hello [1, 2, 3]


## Types

Every value in Julia has a type:

In [8]:
typeof(1.0)

Float64

In [9]:
typeof(1)

Int64

In [10]:
typeof(π)

Irrational{:π}

In [11]:
typeof([1, 2, 3])

Array{Int64,1}

You can create your own types to organize your data:

In [12]:
struct Person
  name::String
end

alice = Person("Alice")

Person("Alice")

Julia's types are extremely lightweight, and user-defined types are *exactly* as performant as anything built-in:

In [13]:
sizeof(Person) == sizeof(Ptr{String})

true

## Multiple Dispatch

Julia does not have classes like Java, Python, or C++. Instead, code is organized around *multiple dispatch*, where the compiler chooses the appropriate method of a given function based on the types of *all* of its input arguments. 

For more, see: [The Unreasonable Effectiveness of Multiple Dispatch (Stefan Karpinski, JuliaCon 2019)](https://www.youtube.com/watch?v=kc9HwsxE1OY)

In [14]:
greet(x, y) = println("$x greets $y")

greet (generic function with 1 method)

In [15]:
alice = Person("alice")
bob = Person("bob")

greet(alice, bob)

Person("alice") greets Person("bob")


Currently there is only one `greet()` function, and it will work on `x` and `y` of any type:

In [16]:
greet([1, 2, 3], "hello world")

[1, 2, 3] greets hello world


We can use abstract types to organize the behavior of related types:

In [17]:
abstract type Animal end

struct Cat <: Animal
    name::String
end

We've already defined `greet(x, y)` for any `x` and `y`, but we can add another definition for a more specific set of input types.

We can be as specific or as general as we like with the argument types:

In [18]:
greet(x::Person, y::Animal) = println("$x pats $y")

greet (generic function with 2 methods)

In [19]:
greet(x::Cat, y) = println("$x meows at $y")

greet (generic function with 3 methods)

Julia will always pick the *most specific* method that matches the provided function arguments.

In [20]:
fluffy = Cat("fluffy")

greet(alice, fluffy)

Person("alice") pats Cat("fluffy")


In [21]:
greet(fluffy, alice)

Cat("fluffy") meows at Person("alice")


In [22]:
struct Dog <: Animal
    name::String
end

greet(x::Dog, y) = println("$x barks at $y")

greet(x::Dog, y::Person) = println("$x licks $y's face")

greet(x::Dog, y::Dog) = println("$x sniffs $y's butt")

greet (generic function with 6 methods)

In [23]:
fido = Dog("fido")
rex = Dog("rex")

greet(alice, fido)

Person("alice") pats Dog("fido")


In [24]:
greet(fido, fluffy)

Dog("fido") barks at Cat("fluffy")


In [25]:
greet(fido, bob)

Dog("fido") licks Person("bob")'s face


In [26]:
greet(fido, rex)

Dog("fido") sniffs Dog("rex")'s butt


If you want to know which `greet` method will be called for a given set of arguments, you can use `@which` to check:

In [27]:
@which greet(fido, fluffy)

greet(x::Dog, y) in Main at In[22]:5

You can list all of the methods of a given function with `methods`:

In [28]:
methods(greet)

# 6 methods for generic function "greet":
[1] greet(x::Dog, y::Dog) in Main at In[22]:9
[2] greet(x::Dog, y::Person) in Main at In[22]:7
[3] greet(x::Person, y::Animal) in Main at In[18]:1
[4] greet(x::Cat, y) in Main at In[19]:1
[5] greet(x::Dog, y) in Main at In[22]:5
[6] greet(x, y) in Main at In[14]:1

It is possible to extend a function with methods later:

In [29]:
greet(x::Dog, y::Cat) = println("$x barks at $y")

greet (generic function with 7 methods)

In [30]:
@which greet(fido, fluffy)

greet(x::Dog, y::Cat) in Main at In[29]:1

## Using Packages

Julia has a built-in package manager called `Pkg`. It handles installing packages and managing all your package environments. 

A package *environment* represents a single set of installed packages. Let's activate the environment for this talk:

In [31]:
using Pkg
Pkg.activate(".")

 Activating environment at `C:\projects\julia-intro2\Project.toml`


(this is similar to `source venv/bin/activate` in a Python virtual environment)

We can install a package in our current environment. This will only affect that environment, so we can safely do this without breaking any other Julia projects we might be working on:

In [32]:
Pkg.add("Colors")

   Updating registry at `C:\Users\Schäffer Krisztián\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
No Changes to `C:\projects\julia-intro2\Project.toml`
No Changes to `C:\projects\julia-intro2\Manifest.toml`


The `Project.toml` file gives a concise description of the packages we've added to this environment:

In [33]:
run(`cat Project.toml`)

[deps]
Colors = "5ae59095-9a9b-59fe-a467-6f913c188581"
HTTP = "cd3eb016-35fb-5094-929b-558a96fad6f3"
ProgressMeter = "92933f4c-e287-5a05-a399-4b506db050ca"


Process(`cat Project.toml`, ProcessExited(0))

The package manager also generates a complete manifest of every package that is installed, including all the transitive dependencies and their versions. You can use this to reproduce a given package environment exactly:

In [34]:
run(`cat Manifest.toml`)

# This file is machine-generated - editing it directly is not advised

[[Artifacts]]
deps = ["Pkg"]
git-tree-sha1 = "c30985d8821e0cd73870b17b0ed0ce6dc44cb744"
uuid = "56f22d72-fd6d-98f1-02f0-08ddc0907c33"
version = "1.3.0"

[[Base64]]
uuid = "2a0f44e3-6c83-55bd-87e4-b1978d98bd5f"

[[ColorTypes]]
deps = ["FixedPointNumbers", "Random"]
git-tree-sha1 = "4bffea7ed1a9f0f3d1a131bbcd4b925548d75288"
uuid = "3da002f7-5984-5a60-b8a6-cbb66c0b333f"
version = "0.10.9"

[[Colors]]
deps = ["ColorTypes", "FixedPointNumbers", "InteractiveUtils", "Reexport"]
git-tree-sha1 = "ac5f2213e56ed8a34a3dd2f681f4df1166b34929"
uuid = "5ae59095-9a9b-59fe-a467-6f913c188581"
version = "0.12.6"

[[Dates]]
deps = ["Printf"]
uuid = "ade2ca70-3891-5945-98fb-dc099432e06a"

[[Distributed]]
deps = ["Random", "Serialization", "Sockets"]
uuid = "8ba89e20-285c-5b6f-9357-94700520ee1b"

[[FixedPointNumbers]]
deps = ["Statistics"]
git-tree-sha1 = "335bfdceacc84c5cdf16aadc768aa5ddfc5383cc"
uuid = "53c48c17-4a7d-5ca2-90c5-79b7896ee

Process(`cat Manifest.toml`, ProcessExited(0))

## Example: Representing Colors and Images with Colors.jl

Let's take a tour of one of my favorite packages, Colors.jl, and show off some feature's of Julia's arrays along the way. 

We can load a package from the current environment with `using`:

In [35]:
using Colors: RGB  # For now, just bring the `RGB` name into scope

In [36]:
RGB(1, 0, 0)

The `RGB` type from Colors.jl knows how to render itself as an actual colored `div` when running in Jupyter. We can also print its value as a string if we want:

In [37]:
print(RGB(1, 0, 0))

RGB{N0f8}(1.0,0.0,0.0)

Julia arrays are fully generic, so we can create an array of colors:

In [38]:
C = [RGB(i, j, 0) for i in 0:0.1:1, j in 0:0.1:1]

In [39]:
typeof(C)

Array{RGB{Float64},2}

C is an array like any other, so we can index into it and slice it:

In [40]:
C[8, 2]

In [41]:
C[1, :]

Let's pull out the red channel from our image `C`:

In [42]:
using Colors: red, green, blue

If you don't know what a function does, you can use the `?` operator to access its docstring:

In [43]:
?red

search: reduce redisplay redirect_stdin redirect_stdout redirect_stderr



`red(c)` returns the red component of an `AbstractRGB` opaque or transparent color.


In [44]:
red(C[1, 1])

0.0

### Broadcasting

To get the red channel of each element of `C`, we can use *broadcasting*. The syntax `f.(x)` applies the function `f` to each element of `x`:

In [45]:
red.(C)

11×11 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1
 0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.2
 0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3
 0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4
 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
 0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6  0.6
 0.7  0.7  0.7  0.7  0.7  0.7  0.7  0.7  0.7  0.7  0.7
 0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8
 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

That's not very visual. Let's render that red channel as a grayscale image:

In [46]:
using Colors: Gray

Gray.(red.(C))

Julia's broadcasting provides guaranteed *loop fusion*. That means that if you do `Gray.(red.(x))`, the language guarantees that it will do only one loop over the elements of `x`, computing `Gray(red(x_i))` for each `x_i` in `x`. 

See https://julialang.org/blog/2017/01/moredots/ for more. 

# Julia is Fast

In [47]:
"""
Naive implementation of sum. Works for any iterable `x` with any element type.
"""
function my_sum(x)
    result = zero(eltype(x))
    for element in x
        result += element
    end
    return result
end

my_sum

And let's create some data to test with:

In [48]:
data = rand(Float64, 10^7)

10000000-element Array{Float64,1}:
 0.955874381165311
 0.7892037691988727
 0.8377818168443907
 0.7596971505424572
 0.855967834817577
 0.7402470423768226
 0.7138547046271644
 0.8107907947811854
 0.23968760717119464
 0.7878905142400041
 0.8327581914433315
 0.9497955745851736
 0.3322496962000745
 ⋮
 0.9945901978036156
 0.046206643676294235
 0.44316343266757996
 0.36863264965157394
 0.29075831656643736
 0.22479356031821895
 0.5730577647462607
 0.0390191937137796
 0.9750027532248191
 0.4754596118932033
 0.2885563598851284
 0.828817482017641

To measure the performance of `my_sum`, we'll use the BenchmarkTools.jl package. 

In [49]:
using BenchmarkTools

In [50]:
@benchmark my_sum($data)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.177 ms (0.00% GC)
  median time:      9.253 ms (0.00% GC)
  mean time:        9.284 ms (0.00% GC)
  maximum time:     10.532 ms (0.00% GC)
  --------------
  samples:          539
  evals/sample:     1

In this case, we only care about the minimum time. The `@btime` macro is a shorthand to print just that minimum time:

In [51]:
@code_native debuginfo=:none my_sum(data)

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	movq	8(%rcx), %rax
	testq	%rax, %rax
	jle	L63
	movq	(%rcx), %rcx
	vxorpd	%xmm0, %xmm0, %xmm0
	vaddsd	(%rcx), %xmm0, %xmm0
	cmpq	$1, %rax
	je	L61
	movl	$1, %edx
	nopw	%cs:(%rax,%rax)
	nopl	(%rax)
L48:
	vaddsd	(%rcx,%rdx,8), %xmm0, %xmm0
	incq	%rdx
	cmpq	%rax, %rdx
	jb	L48
L61:
	popq	%rbp
	retq
L63:
	vxorps	%xmm0, %xmm0, %xmm0
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)
	nop


In [52]:
@btime sum($data)

  3.529 ms (0 allocations: 0 bytes)


5.000389459111943e6

What's going on? Is the `sum()` function using some built-in behavior we don't have access to?

Nope--we can achieve that result easily with a few modifications:

In [53]:
function my_fast_sum(x)
    result = zero(eltype(x))
    
    # `@inbounds` is a macro which disables all bounds checking within a given block. 
    #
    # `@simd` enables additional vector operations by indicating that it is OK to potentially
    # evaluate the loop out-of-order. 
    @inbounds @simd for element in x
        result += element
    end
    result
end

my_fast_sum (generic function with 1 method)

In [54]:
@btime my_fast_sum($data)

  3.526 ms (0 allocations: 0 bytes)


5.000389459111899e6

In [55]:
@code_native debuginfo=:none my_fast_sum(data)

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	movq	8(%rcx), %r8
	testq	%r8, %r8
	jle	L33
	movq	(%rcx), %rcx
	cmpq	$16, %r8
	jae	L39
	vxorpd	%xmm0, %xmm0, %xmm0
	xorl	%edx, %edx
	jmp	L160
L33:
	vxorps	%xmm0, %xmm0, %xmm0
	popq	%rbp
	retq
L39:
	movl	%r8d, %eax
	andl	$15, %eax
	movq	%r8, %rdx
	subq	%rax, %rdx
	vxorpd	%xmm0, %xmm0, %xmm0
	xorl	%eax, %eax
	vxorpd	%xmm1, %xmm1, %xmm1
	vxorpd	%xmm2, %xmm2, %xmm2
	vxorpd	%xmm3, %xmm3, %xmm3
	nopw	%cs:(%rax,%rax)
	nop
L80:
	vaddpd	(%rcx,%rax,8), %ymm0, %ymm0
	vaddpd	32(%rcx,%rax,8), %ymm1, %ymm1
	vaddpd	64(%rcx,%rax,8), %ymm2, %ymm2
	vaddpd	96(%rcx,%rax,8), %ymm3, %ymm3
	addq	$16, %rax
	cmpq	%rax, %rdx
	jne	L80
	vaddpd	%ymm0, %ymm1, %ymm0
	vaddpd	%ymm0, %ymm2, %ymm0
	vaddpd	%ymm0, %ymm3, %ymm0
	vextractf128	$1, %ymm0, %xmm1
	vaddpd	%xmm1, %xmm0, %xmm0
	vpermilpd	$1, %xmm0, %xmm1 # xmm1 = xmm0[1,0]
	vaddsd	%xmm1, %xmm0, %xmm0
	cmpq	%rdx, %r8
	je	L173
	nopw	%cs:(%rax,%rax)
	nop
L160:
	vaddsd	(%rcx,%rdx,8), %xmm0, %xmm0
	incq	%rdx
	cmpq	%rdx, %r8
	jne	L160

With some pretty simple changes, we were able to create a pure-Julia function which is twice as fast as our naive C function while still being clear and completely generic:

In [56]:
my_fast_sum([1, 2.5, π])

6.641592653589793

### What about Numpy or Cython?

* There are faster ways to sum a vector of `double`s in Python than a `for` loop. 
* `numpy.sum()` is just as fast as Julia's `sum()` for large vectors...
* ...but there are some caveats:
  * NumPy is only efficient for a pre-determined set of numeric types. 
  * NumPy cannot be extended without switching into an entirely different programming language, build system, and code environment. 
  * So, if `numpy.sum()` happens to cover the cases you actually need, then go for it!
  * But if you want to be able to write efficient code that does not happen to cover the specific set of functions and types in NumPy, then you need Julia. 

In [57]:
struct Point{T}
    x::T
    y::T
end

function Base.zero(::Type{Point{T}}) where {T} 
    Point{T}(zero(T), zero(T))
end
    
Base.:+(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)

points = [Point(rand(), rand()) for _ in 1:10^7];

In [58]:
@btime my_fast_sum($points)

  6.905 ms (0 allocations: 0 bytes)


Point{Float64}(5.000524400895821e6, 4.99940286074312e6)

# Bonus Features of Julia

## Asynchronous Tasks

Julia supports asynchronous cooperative tasks, with `libuv` providing the backend. These tasks are great for handling operations like IO or network requests:

In [59]:
using Pkg;Pkg.add("HTTP")
using HTTP: request

  Resolving package versions...
No Changes to `C:\projects\julia-intro2\Project.toml`
No Changes to `C:\projects\julia-intro2\Manifest.toml`


In [60]:
for i in 1:5
    @async begin
        println("starting request $i")
        r = request("GET", "https://jsonplaceholder.typicode.com/posts/$i")
        println("got response $i with status $(r.status)")
    end
end
        

starting request 1
starting request 2

## Multi-Threading


Julia also supports parallel and distributed computing (see https://docs.julialang.org/en/v1/manual/parallel-computing/ for more). In addition, Julia 1.3 implemented a new feature, the Parallel Task Run-Time (PATR), which allows for *composable* multi-threading. It is now possible for a parallelized Julia function to call other parallelized code without over-subsubscribing the available processors. See 
https://julialang.org/blog/2019/07/multithreading/ for more. 

In [61]:
using Base.Threads: @spawn


function fib(n::Int)
    if n < 2
        return n
    end
    # `@spawn` creates a new parallel task. Tasks are lightweight and can be
    # created at will. The Julia Parallel Task Run-Time handles scheduling the
    # tasks to native threads in a depth first manner. That means that you can
    # write parallel code which calls other parallel code without over-subscribing
    # your available processors. 
    t = @spawn fib(n - 2)
    return fib(n - 1) + fetch(t)
end 

fib (generic function with 1 method)

starting request 3
starting request 4
starting request 5

In [62]:
fib(10)

55

## Anything Can Be a Value

Julia has no special rules about what can or cannot be assigned to a variable or passed to a function. 

### Functions are Values

A Julia function is a value like any other, so passing functions around and implementing higher-order functions is trivial:

In [63]:
"""
map_reduce: apply `operator` to each element in `array` and reduce pairwise via `reduction`
"""
function map_reduce(operator, reduction, array, initial_value)
    result = initial_value
    for item in array
        result = reduction(result, operator(item))
    end
    result
end

map_reduce

In [64]:
map_reduce(sin, +, [1, 2, 3, 4], 0)

1.1350859243855171

We can define `sum` in terms of `map_reduce`:

In [65]:
fancy_sum(x) = map_reduce(identity, +, x, zero(eltype(x)))

fancy_sum (generic function with 1 method)

The performance is just as good as our hand-written `sum` loop:

To get all the way down to 5ms, we'd need to apply the same `@inbounds` and `@simd` annotations.  

### Types are Values

Types can also be passed around as values and bound to variables with no special rules. This makes implementing factories or constructors easy:

In [66]:
function empty_matrix(T::Type, rows::Integer, cols::Integer)
    zeros(T, rows, cols)
end

empty_matrix (generic function with 1 method)

In [67]:
empty_matrix(Int, 3, 3)

3×3 Array{Int64,2}:
 0  0  0
 0  0  0
 0  0  0

In [68]:
empty_matrix(Point{Float64}, 3, 3)

3×3 Array{Point{Float64},2}:
 Point{Float64}(0.0, 0.0)  Point{Float64}(0.0, 0.0)  Point{Float64}(0.0, 0.0)
 Point{Float64}(0.0, 0.0)  Point{Float64}(0.0, 0.0)  Point{Float64}(0.0, 0.0)
 Point{Float64}(0.0, 0.0)  Point{Float64}(0.0, 0.0)  Point{Float64}(0.0, 0.0)

### Expressions are Values

Even the expressions that representing Julia code are represented as values in Julia. You can create an expression with the `:()` operator, and you can inspect it just like any other object. 

In [69]:
expr = :(1 + 2)

:(1 + 2)

An expression has a `head` indicating what type of expression it is and zero or more `args`:

In [70]:
expr.head

:call

In [71]:
expr.args

3-element Array{Any,1}:
  :+
 1
 2

## Metaprogramming

Since expressions are just values, we can easily write functions to manipulate them:

In [72]:
switch_to_subtraction!(x::Any) = nothing

"""
Change all `+` function calls to `-` function calls. 

<sarcasm>
Great for fixing sign errors in your code!
</sarcasm>
"""
function switch_to_subtraction!(ex::Expr)
    if ex.head == :call && ex.args[1] == :(+)
        ex.args[1] = :(-)
    end
    for i in 2:length(ex.args)
        switch_to_subtraction!(ex.args[i])
    end
end

switch_to_subtraction!

In [73]:
expr = :((1 + 2) * (3 + 4) * sqrt(2))

:((1 + 2) * (3 + 4) * sqrt(2))

In [74]:
switch_to_subtraction!(expr)

expr

:((1 - 2) * (3 - 4) * sqrt(2))

### Macros

A macro is written just like a normal Julia function. The difference is that a macro operates on the *expression* itself, not on its value:

In [75]:
"""
Modify a given expression, replacing all string literals with "cat"
"""
macro more_cats(expr)
    for i in eachindex(expr.args)
        if expr.args[i] isa String
            expr.args[i] = "cat"
        end
    end
    return esc(expr)
end

@more_cats

Macros are always called with the `@` prefix in Julia:

In [76]:
@more_cats println("hello world")

cat


`@macroexpand` shows the code that another macro will generate:

In [77]:
@macroexpand @more_cats println("hello world")

:(println("cat"))

got response 1 with status 200
got response 2 with status 200
got response 3 with status 200
got response 4 with status 200
got response 5 with status 200


# What's Hard to Do in Julia?

What is the compiler team working on making better? https://discourse.julialang.org/t/compiler-work-priorities/17623

What are some subtle problems that the Julia team would like to fix?

[![What's bad about Julia talk](img/bad_about_julia.png)](https://www.youtube.com/watch?v=TPuJsgyu87U)



## Compiler Latency

* The JIT compiler runs each time it sees a function being called with a new input type. 
* That makes the first call to every function slow, since you have to wait for the JIT.
  * This makes Julia awkward to use for things like shell scripts or AWS lambda

## Static Compilation

* To avoid the JIT lag, you can compile a Julia package to a standalone executable using [PackageCompiler.jl](https://github.com/JuliaLang/PackageCompiler.jl), but:
    * This workflow is pretty new, and you may run into interesting bugs
    * The resulting libraries tend to be quite large

## Embedded Computing

* It can be hard to run Julia on memory-limited systems, since you need the compiler living alongside your code. 
* Static compilation can help, but this isn't a well-developed workflow yet.

## Static Analysis

* There are some linting tools for Julia (like the `vscode-julia` extension for Visual Studio Code), but they are not as mature as languages like Python, C, Java, etc.
* Static analysis of Julia is harder, since the language itself is dynamically typed. 
  * To be fair, static analysis of C++ is [undecidable](https://blog.reverberate.org/2013/08/parsing-c-is-literally-undecidable.html) but we still have tools that do a pretty good job most of the time. 

# Useful Julia Tools

## Juno IDE

https://junolab.org/

* Built-in debugger and profiler support
* Inline evaluation of expressions
* Docstring lookup and autocomplete


<img src="img/juno.png" width="700px"></img>

## Julia-VSCode

https://github.com/julia-vscode/julia-vscode

* Code highlighting, snippets, linting, and completions
* Integrated plot and table viewers
* General extension support via the VSCode language server

<img src="img/vscode.png" width="700px"></img>

## Flux.jl

https://fluxml.ai/Flux.jl/stable/

* Flexible library for machine learning built entirely in Julia
* Feed-forward and recurrent neural nets
* Gradients via automatic differentiation
* GPU support via CuArrays.jl

```julia
m = Chain(
  Dense(784, 32, σ),
  Dense(32, 10), softmax
)

loss(x, y) = Flux.mse(m(x), y)
ps = Flux.params(m)

for i in 1:num_training_iters
    Flux.train!(loss, ps, data, opt)
end
```

## DifferentialEquations.jl

https://github.com/SciML/DifferentialEquations.jl

* {stochastic | deterministic | ordinary | partial} differential equations
* Automatic differentiation and sparsity detection
* GPU support
* Sensitivity analysis and parameter estimation
* Access to pure-Julia solvers and existing C and Fortran solvers


<img src="img/DifferentialEquations_Example.png" width="500px"></img>

## DataFrames.jl

https://github.com/JuliaData/DataFrames.jl

* In-memory tabular data
* Joining, indexing, grouping, and split-apply-combine

```julia
julia> using DataFrames

julia> df = DataFrame(A = 1:4, B = ["M", "F", "F", "M"])
4×2 DataFrame
│ Row │ A     │ B      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 1     │ M      │
│ 2   │ 2     │ F      │
│ 3   │ 3     │ F      │
│ 4   │ 4     │ M      │
```

## JuMP.jl

https://github.com/JuliaOpt/JuMP.jl

* Continuous and discrete optimization
* Support for a wide variety of free and commercial solvers
* Efficient high-level language for mathematical programming


Example: Solving a simple model-predictive control problem as a quadratic program ([source](https://github.com/rdeits/DynamicWalking2018.jl/blob/master/notebooks/6.%20Optimization%20with%20JuMP.ipynb)):

[![demo of a simple model-predictive control problem](img/mpc.gif)](https://github.com/rdeits/DynamicWalking2018.jl/blob/master/notebooks/6.%20Optimization%20with%20JuMP.ipynb)



### Cassette.jl

https://www.youtube.com/watch?v=_E2zEzNEy-8

"Overdub" Julia code: 

* Hook into the compiler to dynamically modify the behavior of existing functions in a given *context*
* Building block for debuggers, automatic differentation, and more.

<img src="img/cassette-logo.png" width="600px"/>

# Where To Go Next?

* Download Julia from https://julialang.org/
* Check out the manual at https://docs.julialang.org/en/v1/
* Ask questions on [discourse](https://discourse.julialang.org/) and [slack](https://slackinvite.julialang.org/)
* Find interesting packages on [juliahub](https://juliahub.com/ui/Home)

![triangulated background](img/julia-triangle-background.svg)

[Logo by Cormullion and David P. Saunders](https://nbviewer.jupyter.org/github/dpsanders/JuliaCon2019_tshirt/blob/master/penroseiana.ipynb)